In [1]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array
from sklearn.decomposition import PCA, KernelPCA
from biosppy.signals import ecg  # preprocessing ECG signal
import math

In [7]:
class BioFeature():
    def __init__(self, sample_rate=300,
                 threshold=0.48, segstart=90, segend=110):
        self.sample_rate = sample_rate
        self.threshold = threshold
        self.segstart = segstart  # Q-R interval
        self.segend = segend      # R-S interval

    def fit(self, X, y=None):
        X = check_array(X)
        return self

    def transform(self, X, y=None):
        # X = check_array(X)
        T = 1.0 / self.sample_rate
        bio_feature = np.empty(1)
        for i in range(X.shape[0]):
            if i % 50 == 0:
                print("processed %d" % i)
            Xi = X[i]
            rpeaks = ecg.engzee_segmenter(Xi,
                                          sampling_rate=300,
                                          threshold=self.threshold)
            rpeaks = rpeaks[0]
            # 1. compute the mean of DFT for one QRS wave
            QR_int = self.segstart    # Q-R interval
            RS_int = self.segend      # R-S interval
            m_idx = X.shape[1] - RS_int
            idx = np.where((rpeaks > QR_int) & (rpeaks < m_idx))
            idx = idx[0]
            new_rpeaks = rpeaks[idx]
            _fft = np.zeros(QR_int + RS_int)
            for j in range(len(new_rpeaks)):
                # extract segment around rpeaks
                QRS = Xi[new_rpeaks[j] - QR_int:new_rpeaks[j] + RS_int]
                # compute the mean of DFT
                fft = np.fft.fft(QRS)
                fft_amp = np.absolute(fft)
                if j == 0:
                    _fft = fft_amp
                else:
                    _fft = np.vstack((_fft, fft_amp))
            fft_mean = np.mean(_fft, axis=0)
            if len(new_rpeaks) == 0 or len(new_rpeaks) == 1:
                fft_mean = _fft
                
            # 2. compute the statistics of heart beat
            out = ecg.ecg(signal=Xi, sampling_rate=300,
                          show=False)
            [ts, sig, rpeaks, temp_ts, temp, hr_ts, heart_rate] = out
            heart_stats = [0, 0, 0, 0]
            if len(heart_rate) != 0:
                heart_stats[0] = np.mean(heart_rate)
                heart_stats[1] = np.var(heart_rate)
                heart_stats[2] = np.amin(heart_rate)
                heart_stats[3] = np.amax(heart_rate)
            heart_stats = np.asarray(heart_stats)

            # 3. compute the statistics of R-R interval
            RR_int = []
            for k in range(1, len(rpeaks)):
                RR_int.append(T * (rpeaks[k] - rpeaks[k-1]))
            RR_int = np.asarray(RR_int)
            RR_stats = [0, 0, 0, 0]
            if len(RR_int) != 0:
                RR_stats[0] = np.mean(RR_int)
                RR_stats[1] = np.var(RR_int)
                RR_stats[2] = np.amin(RR_int)
                RR_stats[3] = np.amax(RR_int)
            RR_stats = np.asarray(RR_stats)

            feature = np.hstack((fft_mean, heart_stats))
            feature = np.hstack((feature, RR_stats))
            if i == 0:
                bio_feature = feature
            else:
                bio_feature = np.vstack((bio_feature, feature))
        return bio_feature
fs = BioFeature()

In [8]:
class HRVFeature(BaseEstimator, TransformerMixin):
    """ Adpated from <<van Gent, P. (2016).
    Analyzing a Discrete Heart Rate Signal Using Python.
    A tech blog about fun things with Python and embedded electronics>>"""
    """ feature includes: BPM, IBI, SDNN. SDSD, RMSSD, pNN50, pNN20 """
    def __init__(self, sample_rate=300,
                 start=40, end=50, save_path=None):
        self.sample_rate = sample_rate
        self.segstart = start
        self.segend = end
        self.save_path = save_path

    def fit(self, X, y=None):
        X = check_array(X)
        return self

    def transform(self, X, y=None):
        T = 1.0 / self.sample_rate
        bio_feature = np.empty(1)
        for i in range(X.shape[0]):
            if i % 50 == 0:
                print("processed %d" % i)
            Xi = X[i, :]
            # extract the R-peaks positions
            out = ecg.ecg(signal=Xi, sampling_rate=300,
                          show=False)
            [ts, sig, rpeaks, temp_ts, temp, hr_ts, heart_rate] = out
            # 1. Extract Time Domain Measures
            # Compute the R-R interval (ms)
            RR_int = []
            for k in range(1, len(rpeaks)):
                RR_int.append(1000.0 * T * (rpeaks[k] - rpeaks[k - 1]))
            RR_int = np.asarray(RR_int)
            # Compute the R-R interval difference
            RR_diff = []    # difference between adajacent RR interval
            RR_sqdiff = []  # squared difference of RR interval
            for k in range(1, len(RR_int)):
                m_diff = RR_int[k] - RR_int[k - 1]
                RR_diff.append(abs(m_diff))
                RR_sqdiff.append(math.pow(m_diff, 2))
            RR_diff = np.asarray(RR_diff)
            RR_sqdiff = np.asarray(RR_sqdiff)

            bio_feature = np.zeros(10)
            if len(RR_int) >= 1:
                # 1.ibi measure: the mean of the R-R interval
                ibi = np.mean(RR_int)
                bio_feature[0] = ibi

                # 2.sdn measure: the standard deviation of R-R intervals
                sdnn = np.std(RR_int)
                bio_feature[1] = sdnn

                # 3. max of the R-R interval
                bio_feature[2] = np.amax(RR_int)

                # data = pd.Series(RR_int)
                # arma_mod4 = sm.tsa.ARMA(data, (4,0)).fit()
                # bio_feature[3] = arma_mod4.params[0]
                # bio_feature[4] = arma_mod4.params[1]
                # bio_feature[5] = arma_mod4.params[2]
                # bio_feature[6] = arma_mod4.params[3]

                if len(RR_diff) >= 1:
                    # 5.sdsd measure: the standard deviation of the R-R diff
                    sdsd = np.std(RR_diff)
                    bio_feature[3] = sdsd

                    # 6. the mean , min, max of the R-R difference
                    bio_feature[4] = np.mean(RR_diff)
                    bio_feature[5] = np.amax(RR_diff)

                    # 4.rsmd measure: the root mean square of R-R differences
                    rmssd = np.sqrt(np.mean(RR_sqdiff))
                    bio_feature[6] = rmssd

                    # 5. pnn20/50: the percentage of differences > 50/20 ms
                    nn20 = [x for x in RR_diff if (x > 20)]
                    nn50 = [x for x in RR_diff if (x > 50)]
                    pnn20 = float(len(nn20)) / float(len(RR_diff))
                    pnn50 = float(len(nn50)) / float(len(RR_diff))
                    bio_feature[7] = pnn20
                    bio_feature[8] = pnn50

                # 6. bpm measure: the average heart rate per minute
                # bpm = np.mean(heart_rate)
                bpm = 60000 / np.mean(RR_int)
                bio_feature[9] = bpm

                # 7. extract the QRS waves and use the mean as feature
                QR_int = self.segstart    # Q-R interval
                RS_int = self.segend      # R-S interval
                m_idx = X.shape[1] - RS_int
                idx = np.where((rpeaks > QR_int) & (rpeaks < m_idx))
                idx = idx[0]
                new_rpeaks = rpeaks[idx]
                nQRS = np.zeros(QR_int + RS_int)
                for j in range(len(new_rpeaks)):
                    # extract segment around rpeaks
                    QRS = Xi[new_rpeaks[j] - QR_int:new_rpeaks[j] + RS_int]
                    # QRS = np.absolute(np.fft.fft(QRS))
                    # compute the mean of DFT
                    if j == 0:
                        nQRS = QRS
                    else:
                        nQRS = np.vstack((nQRS, QRS))
                QRS_mean = np.mean(nQRS, axis=0)
                if len(new_rpeaks) == 0 or len(new_rpeaks) == 1:
                    QRS_mean = nQRS
                bio_feature = np.hstack((bio_feature, QRS_mean))

            for j in range(len(bio_feature)):
                if np.isnan(bio_feature[j]):
                    print("Error in extracting sample")
                    print(i)
                    print(":found NaN in bio_feature")
            if i == 0:
                features = bio_feature
            else:
                features = np.vstack((features, bio_feature))
        return features
hrvfs = HRVFeature()

In [11]:
signals = []
with open("data/X_train.csv") as f_train:
    for line in f_train.readlines()[1:]:
        s = list(map(int, line.split(',')[1:]))
        if len(s) < 18155:
            s.extend([0 for x in range(len(s), 18155)])
        signals.append(s)
signals = np.array([np.array(s) for s in signals])
y = np.loadtxt('data/y_train.csv', delimiter=',', skiprows=1, usecols=range(1,2))

In [12]:
bioFeatures = fs.transform(signals)

processed 0
processed 50
processed 100
processed 150
processed 200
processed 250
processed 300
processed 350
processed 400
processed 450
processed 500
processed 550
processed 600
processed 650
processed 700
processed 750
processed 800
processed 850
processed 900
processed 950
processed 1000
processed 1050
processed 1100
processed 1150
processed 1200
processed 1250
processed 1300
processed 1350
processed 1400
processed 1450
processed 1500
processed 1550
processed 1600
processed 1650
processed 1700
processed 1750
processed 1800
processed 1850
processed 1900
processed 1950
processed 2000
processed 2050
processed 2100
processed 2150
processed 2200
processed 2250
processed 2300
processed 2350
processed 2400
processed 2450
processed 2500
processed 2550
processed 2600
processed 2650
processed 2700
processed 2750
processed 2800
processed 2850
processed 2900
processed 2950
processed 3000
processed 3050
processed 3100
processed 3150
processed 3200
processed 3250
processed 3300
processed 3350
pro

In [15]:
hrvFeatures = hrvfs.transform(signals)

processed 0
processed 50
processed 100
processed 150
processed 200
processed 250
processed 300
processed 350
processed 400
processed 450
processed 500
processed 550
processed 600
processed 650
processed 700
processed 750
processed 800
processed 850
processed 900
processed 950
processed 1000
processed 1050
processed 1100
processed 1150
processed 1200
processed 1250
processed 1300
processed 1350
processed 1400
processed 1450
processed 1500
processed 1550
processed 1600
processed 1650
processed 1700
processed 1750
processed 1800
processed 1850
processed 1900
processed 1950
processed 2000
processed 2050
processed 2100
processed 2150
processed 2200
processed 2250
processed 2300
processed 2350
processed 2400
processed 2450
processed 2500
processed 2550
processed 2600
processed 2650
processed 2700
processed 2750
processed 2800
processed 2850
processed 2900
processed 2950
processed 3000
processed 3050
processed 3100
processed 3150
processed 3200
processed 3250
processed 3300
processed 3350
pro

In [14]:
with open("bioFeatures.csv", "w") as f:
    for idx,vals in enumerate(bioFeatures):
        s = str(idx)
        for v in vals:
            s += ",%f" % v
        s += "\n"
        f.write(s)


In [16]:
with open("hrvFeatures.csv", "w") as f:
    for idx,vals in enumerate(hrvFeatures):
        s = str(idx)
        for v in vals:
            s += ",%f" % v
        s += "\n"
        f.write(s)


In [36]:
y = np.loadtxt('data/y_train.csv', delimiter=',', skiprows=1, usecols=range(1,2))

In [68]:
X_add = np.loadtxt('data/rpeakfeature.csv', delimiter=',', skiprows=1, usecols=range(1,9))
newX = []
for i in range(len(X)):
     newX.append(np.concatenate([X[i], X_add[i]]))

In [81]:
X_add2 = hrvfs.transform(np.array([np.array(s) for s in signals]))

processed 0
processed 50
processed 100
processed 150
processed 200
processed 250
processed 300
processed 350
processed 400
processed 450
processed 500
processed 550
processed 600
processed 650
processed 700
processed 750
processed 800
processed 850
processed 900
processed 950
processed 1000
processed 1050
processed 1100
processed 1150
processed 1200
processed 1250
processed 1300
processed 1350
processed 1400
processed 1450
processed 1500
processed 1550
processed 1600
processed 1650
processed 1700
processed 1750
processed 1800
processed 1850
processed 1900
processed 1950
processed 2000
processed 2050
processed 2100
processed 2150
processed 2200
processed 2250
processed 2300
processed 2350
processed 2400
processed 2450
processed 2500
processed 2550
processed 2600
processed 2650
processed 2700
processed 2750
processed 2800
processed 2850
processed 2900
processed 2950
processed 3000
processed 3050
processed 3100
processed 3150
processed 3200
processed 3250
processed 3300
processed 3350
pro

In [82]:
newX2 = []
for i in range(len(X)):
     newX2.append(np.concatenate([newX[i], X_add2[i]]))

In [94]:
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
train_X, eval_X, train_Y, eval_Y = train_test_split(newX2, y, test_size=0.33, random_state=42)
xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_eval = xgb.DMatrix(eval_X, label=eval_Y)

# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.2
param['gamma'] = 1.0
param['max_depth'] = 6
param['silent'] = 1
param['subsample'] = 0.8
param['colsample_bytree'] = 0.9
param['min_child_weight'] = 20
param['num_class'] = 4


watchlist = [(xg_train, 'train'), (xg_eval, 'eval')]
num_round = 80
bst = xgb.train(param, 
                xg_train, 
                num_round, 
                watchlist, 
                feval=lambda y,t: ("f1", f1_score(y, t.get_label(), average='micro')))

# get prediction
pred = bst.predict(xg_eval)
# error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
F1 = f1_score(eval_Y, pred, average='micro')
print('Test error using softmax = {}'.format(F1))

[0]	train-merror:0.203326	eval-merror:0.23505	train-f1:0.796674	eval-f1:0.76495
[1]	train-merror:0.182614	eval-merror:0.213144	train-f1:0.817386	eval-f1:0.786856
[2]	train-merror:0.17853	eval-merror:0.210184	train-f1:0.82147	eval-f1:0.789816
[3]	train-merror:0.173279	eval-merror:0.208999	train-f1:0.826721	eval-f1:0.791001
[4]	train-merror:0.164819	eval-merror:0.206631	train-f1:0.835181	eval-f1:0.793369
[5]	train-merror:0.162777	eval-merror:0.201303	train-f1:0.837223	eval-f1:0.798697
[6]	train-merror:0.155193	eval-merror:0.20071	train-f1:0.844807	eval-f1:0.79929
[7]	train-merror:0.153151	eval-merror:0.199526	train-f1:0.846849	eval-f1:0.800474
[8]	train-merror:0.1479	eval-merror:0.19775	train-f1:0.8521	eval-f1:0.80225
[9]	train-merror:0.144399	eval-merror:0.188869	train-f1:0.855601	eval-f1:0.811131
[10]	train-merror:0.140023	eval-merror:0.190053	train-f1:0.859977	eval-f1:0.809947
[11]	train-merror:0.139148	eval-merror:0.192422	train-f1:0.860852	eval-f1:0.807578
[12]	train-merror:0.129813

In [17]:
testsignals = []
with open("data/X_test.csv") as f_test:
    for line in f_test.readlines()[1:]:
        s = list(map(int, line.split(',')[1:]))
        if len(s) < 18155:
            s.extend([0 for x in range(len(s), 18155)])
        testsignals.append(s)

In [18]:
test_X = fs.transform(np.array([np.array(s) for s in testsignals]))

processed 0
processed 50
processed 100
processed 150
processed 200
processed 250
processed 300
processed 350
processed 400
processed 450
processed 500
processed 550
processed 600
processed 650
processed 700
processed 750
processed 800
processed 850
processed 900
processed 950
processed 1000
processed 1050
processed 1100
processed 1150
processed 1200
processed 1250
processed 1300
processed 1350
processed 1400
processed 1450
processed 1500
processed 1550
processed 1600
processed 1650
processed 1700
processed 1750
processed 1800
processed 1850
processed 1900
processed 1950
processed 2000
processed 2050
processed 2100
processed 2150
processed 2200
processed 2250
processed 2300
processed 2350
processed 2400
processed 2450
processed 2500
processed 2550
processed 2600
processed 2650
processed 2700
processed 2750
processed 2800
processed 2850
processed 2900
processed 2950
processed 3000
processed 3050
processed 3100
processed 3150
processed 3200
processed 3250
processed 3300
processed 3350
pro

In [19]:
with open("bioFeatures_test.csv", "w") as f:
    for idx,vals in enumerate(test_X):
        s = str(idx)
        for v in vals:
            s += ",%f" % v
        s += "\n"
        f.write(s)


In [69]:
addtest_X = np.loadtxt('data/rpeakfeature_test.csv', delimiter=',', skiprows=1, usecols=range(1,9))
test_X_new = []
for i in range(len(test_X)):
     test_X_new.append(np.concatenate([test_X[i], addtest_X[i]]))

In [20]:
test_X2 = hrvfs.transform(np.array([np.array(s) for s in testsignals]))

processed 0
processed 50
processed 100
processed 150
processed 200
processed 250
processed 300
processed 350
processed 400
processed 450
processed 500
processed 550
processed 600
processed 650
processed 700
processed 750
processed 800
processed 850
processed 900
processed 950
processed 1000
processed 1050
processed 1100
processed 1150
processed 1200
processed 1250
processed 1300
processed 1350
processed 1400
processed 1450
processed 1500
processed 1550
processed 1600
processed 1650
processed 1700
processed 1750
processed 1800
processed 1850
processed 1900
processed 1950
processed 2000
processed 2050
processed 2100
processed 2150
processed 2200
processed 2250
processed 2300
processed 2350
processed 2400
processed 2450
processed 2500
processed 2550
processed 2600
processed 2650
processed 2700
processed 2750
processed 2800
processed 2850
processed 2900
processed 2950
processed 3000
processed 3050
processed 3100
processed 3150
processed 3200
processed 3250
processed 3300
processed 3350
pro

In [21]:
with open("hrvFeatures_test.csv", "w") as f:
    for idx,vals in enumerate(test_X2):
        s = str(idx)
        for v in vals:
            s += ",%f" % v
        s += "\n"
        f.write(s)


In [85]:
test_X_new2 = []
for i in range(len(test_X)):
     test_X_new2.append(np.concatenate([test_X_new[i], test_X2[i]]))

In [95]:
xg_test = xgb.DMatrix(test_X_new2)
y_pred = bst.predict(xg_test)
f = open("submission.csv", "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,y_pred[i]))
f.close()